# 2.1 Dataset Artigos SciELO

## Imports

In [1]:
import os

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

## Código

In [2]:
path_artigos = os.path.join(os.getcwd(), 'artigos_brutos', 'SciELO')

In [3]:
# contagem de artigos
soma = sum(len(os.listdir(os.path.join(path_artigos, a))) for a in os.listdir(path_artigos))
print(f'Total de artigos: {soma}')

Total de artigos: 1681


### Criando o DataSet

In [4]:
dict_df = {'titulo': [], 'seção': [], 'ano': [],  'doi_id': [], 'texto_html': []}

for ano in tqdm(os.listdir(path_artigos)):
    path_abs = os.path.join(path_artigos, ano)

    for arquivo in os.listdir(path_abs):
        arq_path = os.path.join(path_abs, arquivo)

        with open(arq_path, 'r', encoding='utf8') as arq:
            texto_html = arq.read()

        soup = BeautifulSoup(texto_html, 'lxml')

        try:
            doi_id = soup.find('a', attrs={'class': '_doi'}).text.strip().removeprefix('https://doi.org/')
        except AttributeError:
            doi_id = None

        try:
            titulo = ' '.join(soup.find('h1', attrs={'class': 'article-title'}).text.split())
            if not titulo:
                titulo = 'SEM TÍTULO'
        except AttributeError:
            titulo = 'SEM TÍTULO'

        try:
            seção = soup.find('span', attrs={'class': '_articleBadge'}).text.strip().lower()
            if not seção:
                seção = 'SEM CATEGORIA'
        except AttributeError:
            seção = 'SEM CATEGORIA'

        dict_df['titulo'].append(titulo)
        dict_df['doi_id'].append(doi_id)
        dict_df['texto_html'].append(texto_html)
        dict_df['seção'].append(seção)
        dict_df['ano'].append(int(ano))


  0%|          | 0/22 [00:00<?, ?it/s]

In [5]:
# salvando o dataset em feather
df = pd.DataFrame(dict_df)
df['ano'] = df['ano'].astype(np.uint16)
df['seção'] = df['seção'].replace({'artigo gerais': 'artigos gerais',
                                   'resenhas de livro': 'resenhas de livros',
                                   'notas e discussõoes': 'notas e discussões'})

In [10]:
df.to_feather('scielo_dataset.feather')